In [1]:
import pandas as pd
import nfl_data_py as nfl
import numpy
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [2]:
import streamlit as st

from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()

In [3]:
df1 = nfl.import_pbp_data([2022], downcast=True, cache=False, alt_path=None)

2022 done.
Downcasting floats.


In [4]:
df1.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

In [5]:
df1['play_type'] = df1.apply(lambda row: 'pass' if (row['qb_scramble'] == 1) else row['play_type'], axis=1)

In [6]:
plays = df1[(df1['play_type'] == 'run') | (df1['play_type'] == 'pass')]

In [7]:
regular_season = plays[plays['season_type'] == 'REG']

In [8]:
def downs_bin(row):
    if row['ydstogo'] <= 3:
        return 'Short (<=3)'
    elif 3 < row['ydstogo'] <= 8:
        return 'Medium (4-8)'
    elif 8 < row['ydstogo'] <= 12:
        return 'Long (9-12)'
    else:
        return 'Very Long (13+)'

In [9]:
regular_season['downs_label'] = regular_season.apply(lambda row: downs_bin(row), axis=1)

C:\Users\PC\AppData\Local\Temp\ipykernel_17448\638280399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regular_season['downs_label'] = regular_season.apply(lambda row: downs_bin(row), axis=1)


In [11]:
streamlit_data = regular_season[['game_id',
                                 'home_team',
                                 'away_team',
                                 'season_type',
                                 'week',
                                 'posteam',
                                 'posteam_type',
                                 'defteam',
                                 'side_of_field',
                                 'yardline_100',
                                 'game_date',
                                 'quarter_seconds_remaining',
                                 'half_seconds_remaining',
                                 'game_seconds_remaining',
                                 'game_half',
                                 'quarter_end',
                                 'drive',
                                 'qtr',
                                 'down',
                                 'goal_to_go',
                                 'time',
                                 'yrdln',
                                 'ydstogo',
                                 'ydsnet',
                                 'play_type',
                                 'yards_gained',
                                 'shotgun',
                                 'no_huddle',
                                 'pass_length',
                                 'air_yards',
                                 'run_location',
                                 'run_gap',
                                 'posteam_timeouts_remaining',
                                 'defteam_timeouts_remaining',
                                 'fg_prob',
                                 'td_prob',
                                 'epa',
                                 'air_epa',
                                 'yac_epa',
                                 'comp_air_epa',
                                 'third_down_converted',
                                 'third_down_failed',
                                 'fourth_down_converted',
                                 'fourth_down_failed',
                                 'incomplete_pass',
                                 'interception',
                                 'safety',
                                 'tackled_for_loss',
                                 'qb_hit',
                                 'sack',
                                 'touchdown',
                                 'pass_touchdown',
                                 'rush_touchdown',
                                 'fumble',
                                 'complete_pass',
                                 'time_of_day',
                                 'stadium',
                                 'weather',
                                 'roof',
                                 'surface',
                                 'temp',
                                 'wind',
                                 'offense_formation',
                                 'offense_personnel',
                                 'defenders_in_box',
                                 'defense_personnel',
                                 'number_of_pass_rushers',
                                 'downs_label']]

In [12]:
streamlit_data.head()

,play_id,game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,play_type,yards_gained,shotgun,no_huddle,pass_length,air_yards,run_location,run_gap,posteam_timeouts_remaining,defteam_timeouts_remaining,fg_prob,td_prob,epa,air_epa,yac_epa,comp_air_epa,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,interception,safety,tackled_for_loss,qb_hit,sack,touchdown,pass_touchdown,rush_touchdown,fumble,complete_pass,time_of_day,stadium,weather,roof,surface,temp,wind,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,downs_label
2,68.0,2022_01_BAL_NYJ,NYJ,BAL,REG,1,NYJ,home,BAL,NYJ,78.0,2022-09-11,896.0,1796.0,3596.0,Half1,0.0,1.0,1.0,1.0,0.0,14:56,NYJ 22,10.0,14.0,run,19.0,0.0,0.0,None,NaN,left,end,3.0,3.0,0.200609,0.378776,1.468819,NaN,NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17:06:38,MetLife Stadium,"Rain and mid 70s Temp: 73° F, Humidity: 79%, W...",outdoors,fieldturf,NaN,NaN,SINGLEBACK,"1 RB, 2 TE, 2 WR",7.0,"3 DL, 4 LB, 4 DB",NaN,Long (9-12)
3,89.0,2022_01_BAL_NYJ,NYJ,BAL,REG,1,NYJ,home,BAL,NYJ,59.0,2022-09-11,869.0,1769.0,3569.0,Half1,0.0,1.0,1.0,1.0,0.0,14:29,NYJ 41,10.0,14.0,pass,0.0,1.0,1.0,short,0.0,None,None,3.0,3.0,0.245590,0.465806,-0.492192,-0.492192,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17:07:04,MetLife Stadium,"Rain and mid 70s Temp: 73° F, Humidity: 79%, W...",outdoors,fieldturf,NaN,NaN,SHOTGUN,"1 RB, 2 TE, 2 WR",6.0,"3 DL, 4 LB, 4 DB",4.0,Long (9-12)
4,115.0,2022_01_BAL_NYJ,NYJ,BAL,REG,1,NYJ,home,BAL,NYJ,59.0,2022-09-11,865.0,1765.0,3565.0,Half1,0.0,1.0,1.0,2.0,0.0,14:25,NYJ 41,10.0,14.0,run,5.0,1.0,1.0,None,NaN,left,end,3.0,3.0,0.234310,0.433619,-0.325931,NaN,NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17:07:28,MetLife Stadium,"Rain and mid 70s Temp: 73° F, Humidity: 79%, W...",outdoors,fieldturf,NaN,NaN,SINGLEBACK,"1 RB, 2 TE, 2 WR",7.0,"3 DL, 4 LB, 4 DB",NaN,Long (9-12)
5,136.0,2022_01_BAL_NYJ,NYJ,BAL,REG,1,NYJ,home,BAL,NYJ,54.0,2022-09-11,841.0,1741.0,3541.0,Half1,0.0,1.0,1.0,3.0,0.0,14:01,NYJ 46,5.0,14.0,pass,0.0,1.0,1.0,short,0.0,None,None,3.0,3.0,0.215781,0.418190,-2.402200,-1.672078,-0.730123,0.000000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,17:07:52,MetLife Stadium,"Rain and mid 70s Temp: 73° F, Humidity: 79%, W...",outdoors,fieldturf,NaN,NaN,SHOTGUN,"1 RB, 2 TE, 2 WR",7.0,"3 DL, 4 LB, 4 DB",4.0,Medium (4-8)
7,202.0,2022_01_BAL_NYJ,NYJ,BAL,REG,1,BAL,away,NYJ,BAL,72.0,2022-09-11,822.0,1722.0,3522.0,Half1,0.0,2.0,1.0,1.0,0.0,13:42,BAL 28,10.0,21.0,pass,4.0,0.0,0.0,short,-4.0,None,None,3.0,3.0,0.223840,0.363601,0.075127,-1.309201,1.384328,-1.309201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17:12:11,MetLife Stadium,"Rain and mid 70s Temp: 73° F, Humidity: 79%, W...",outdoors,fieldturf,NaN,NaN,SINGLEBACK,"2 RB, 1 TE, 2 WR",7.0,"4 DL, 3 LB, 4 DB",3.0,Long (9-12)
